# Soundscapes

Artificially generated soundscapes created from data, representing a range of cities.

In [3]:
import csv
import itertools
import time
from dataclasses import dataclass

import mido

Data is stored as a mapping of *cities* to a set of *tracks*, each containing *ticks*, each containing *messages*. Each track represents a single data source, and each tick represents an instant in time. Tracks can be looped, not necessarily in sync with each other.

Ticks additionally contain a list of "cleanup messages," which will stop any notes currently being played in case the user wishes to terminate the audio.

In [4]:
CITIES = ["LAX", "ANC", "BOS"]

@dataclass
class Tick:
    messages: list[mido.Message]
    cleanup: list[mido.Message]
    
    @classmethod
    def merge(cls, *ticks):
        messages = []
        cleanup = []
        
        for tick in ticks:
            messages += tick.messages
            cleanup += tick.cleanup
        
        return cls(messages, cleanup)

TRACKS: dict[str, dict[str, list[Tick]]] = {
    city: {
        "weather": [],
        "tides": [],
    } for city in CITIES
}

print(TRACKS)

{'LAX': {'weather': [], 'tides': []}, 'ANC': {'weather': [], 'tides': []}, 'BOS': {'weather': [], 'tides': []}}


We'll want to map data to notes on a musical scale. We can generate lookup tables (index -> note number) for a few common ones.

In [5]:
def extend_scale(scale):
    return list(
        note for note in itertools.chain(
            *([i + offset for offset in scale] for i in range(0, 127, 12))
        )
        if note in range(0, 127)
    )

SCALES = {
    "chromatic": list(range(127)),
    "major": extend_scale([0, 2, 4, 5, 7, 9, 11]),
    "minor": extend_scale([0, 2, 3, 5, 7, 8, 10]),
    "pentatonic": extend_scale([0, 2, 4, 7, 9]),
}

print(SCALES["pentatonic"])

[0, 2, 4, 7, 9, 12, 14, 16, 19, 21, 24, 26, 28, 31, 33, 36, 38, 40, 43, 45, 48, 50, 52, 55, 57, 60, 62, 64, 67, 69, 72, 74, 76, 79, 81, 84, 86, 88, 91, 93, 96, 98, 100, 103, 105, 108, 110, 112, 115, 117, 120, 122, 124]


### Weather Data

Data is sourced from NOAA, through their free [Climate Data Online](https://www.ncdc.noaa.gov/cdo-web/) service.

In [6]:
with open('sources/weather.csv', 'r') as f:
    weather_data = list(csv.reader(f))
  
weather_header = weather_data[0]
weather_data = weather_data[1:]

print(weather_header)

['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09']


This includes data from the following stations:

| Station ID | City |
| --- | --- |
| USW00023174 | Los Angeles |
| USW00026451 | Anchorage |
| USW00014739 | Boston |

Each station reports the following data:

* Average Wind
* Precipitation
* Average Temperature

In [7]:
from dataclasses import dataclass

@dataclass
class WeatherReport:
    station: str
    date: str
    wind: float
    precip: float
    temp: float

In [8]:
weather_reports = {}

WEATHER_STATIONS = {
    "USW00023174": "LAX",
    "USW00026451": "ANC",
    "USW00014739": "BOS",
}

for row in weather_data:
    report = WeatherReport(
        WEATHER_STATIONS[row[weather_header.index("STATION")]],
        row[weather_header.index("DATE")],
        float(row[weather_header.index("AWND")]), # average wind
        float(row[weather_header.index("PRCP")]), # precipitation
        float(row[weather_header.index("TMAX")]) # max temp
    )
    if report.station not in weather_reports:
        weather_reports[report.station] = []
    weather_reports[report.station].append(report)

print(*[f"{station}: {len(reports)} reports" for station, reports in weather_reports.items()], sep='\n')

LAX: 365 reports
ANC: 365 reports
BOS: 365 reports


Audio is generated as MIDI files by traversing chronologically through the readings.

Temperature maps to note pitch, using a chromatic scale.

Wind speed maps to CC 21, and precipitation maps to CC 22.

In [9]:
for station, reports in weather_reports.items():
    track = TRACKS[station]["weather"]
    
    prev_note = None
    
    for report in reports:
        note = SCALES["pentatonic"][int(round(report.temp * 3/5 - 8))]
        
        tick = []
        
        tick.append(mido.Message("control_change", channel=0, control=22, value=int(min(round(report.precip * 50), 127))))

        
        tick.append(mido.Message("note_on", note=note, velocity=127))
        if prev_note is not None and prev_note != note:
            tick.append(mido.Message("note_off", note=prev_note, velocity=127))
        
        tick.append(mido.Message("control_change", channel=0, control=21, value=int(round(report.wind * 127/30))))
        
        prev_note = note
        
        track.append(Tick(tick, [mido.Message("note_off", note=note, velocity=127)]))

### Tidal Data

This data comes from NOAA's [Tides and Currents](https://tidesandcurrents.noaa.gov/) service, using the following marine stations:

| Station ID | City |
| --- | --- |
| 9410660 | Los Angeles |
| 9455920 | Anchorage |
| 8443970 | Boston |

In [10]:
MARINE_STATIONS = {
    "9410660": "LAX",
    "9455920": "ANC",
    "8443970": "BOS",
}

tidal_reports = {}

for station in MARINE_STATIONS:
    with open(f'sources/tides/{station}.csv', 'r') as f:
        data = list(csv.reader(f))
        header = data[0]
        data = data[1:]
        tidal_reports[station] = [float(row[header.index(" Water Level")]) for row in data]
        
print(*[f"{station}: {len(reports)} reports" for station, reports in tidal_reports.items()], sep='\n')

9410660: 8760 reports
9455920: 8713 reports
8443970: 8760 reports


In [11]:
tidal_averages = {
  station: sum(reports) / len(reports)
  for station, reports in tidal_reports.items()
}

tidal_reports = {
  station: [report - tidal_averages[station] for report in reports]
  for station, reports in tidal_reports.items()
}

In [12]:
for station, reports in tidal_reports.items():
    track = TRACKS[MARINE_STATIONS[station]]["tides"]
    
    prev_note = None
    
    for report in reports:
        # note = SCALES["pentatonic"][int(round(report * 2/3))]
        note = 10
        
        tick = []

        tick.append(mido.Message("note_on", channel=1, note=note, velocity=127))
        
        scaled = max(min(int(round(report * 8 + 64)), 127), 0)

        tick.append(mido.Message("control_change", channel=1, control=23, value=scaled))
        
        if prev_note is not None and prev_note != note:
            tick.append(mido.Message("note_off", channel=1, note=prev_note, velocity=127))
        
        prev_note = note
        
        track.append(Tick(tick, [mido.Message("note_off", channel=1, note=note, velocity=127)]))

## Playing Tracks

In [13]:
def play_midi(city):
    port = mido.open_output("Soundscapes by breq", virtual=True)
    
    tick_count = 0
    tick = None
    
    try:
        while True:
            try:
                tick = Tick.merge(*[track[tick_count % len(track)] for track in TRACKS[city].values() if tick_count < len(track)])
                
                for message in tick.messages:
                    port.send(message)
                    
                tick_count += 1
                    
                time.sleep(0.1)
            finally:
                if tick is not None:          
                    for message in tick.cleanup:
                        port.send(message)
    except KeyboardInterrupt:
        pass
    finally:
        port.close()

In [14]:
play_midi("ANC")